<a href="https://colab.research.google.com/github/RazLands/OddEvenClassification/blob/master/OddEven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount to Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Import relevant modules

In [0]:
from keras.datasets import mnist
from keras.models import Sequential, Model
import keras.layers as layers
from keras.utils import to_categorical
from keras import optimizers
from keras.models import model_from_json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from plot_confusion_matrix import plot_confusion_matrix
from keras.layers import Dense, Flatten, Activation


## Load MNist digits dataset

In [0]:
# Load MNist dataset of digits images 0-9
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_oe = x_train
x_test_oe = x_test

# Create Odd and Even array
y_train_oe = y_train%2
y_test_oe = y_test%2

# Make categorized tables
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train_oe = to_categorical(y_train_oe)
y_test_oe = to_categorical(y_test_oe)



# Print shapes
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print("")
print('x_train ODD/EVEN shape:', x_train_oe.shape)
print('x_test ODD/EVEN shape:', x_test_oe.shape)
print('y_train ODD/EVEN shape:', y_train_oe.shape)
print('y_test ODD/EVEN shape:', y_test_oe.shape)

# Hyper Parameters initialization

In [0]:
num_of_clss = 10          # number of classes
num_of_clss_oe = 2
hidden_size =        30    # number of neurons in the hidden layer
lr =        0.0001        # learning rate 
beta_1 =   0.9              # beta 1 - for adam optimizer
beta_2 =      0.99           # beta 2 - for adam optimizer
epsilon =         1e-8       # epsilon - for adam optimizer
epochs =               50  # number of epochs 
bs =                  64   # batch size


# Model Configuration

10 Classes classification - 0-9 digits

In [0]:
model = Sequential()

model.add(Flatten())
model.add(Dense(hidden_size))
model.add(Activation('relu'))
model.add(Dense(hidden_size))
model.add(Activation('sigmoid'))
model.add(Dense(num_of_clss))
model.add(Activation('softmax'))


2 Classes classification - Odd and Even

In [0]:
model_oe = Sequential()

model_oe.add(Flatten())
model_oe.add(Dense(hidden_size))
model_oe.add(Activation('relu'))
model_oe.add(Dense(hidden_size))
model_oe.add(Activation('sigmoid'))
model_oe.add(Dense(num_of_clss_oe))
model_oe.add(Activation('sigmoid'))
# model_oe.add(Activation('softmax')) # <--- Maybe we should try sigmoid instead of softmax here for binary classification


# Model training and show results

Train the 10 classes model

In [0]:
# Define the optimizer and compile the model
adam = optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model, iterating on the data in batches of 64 samples
history = model.fit(x_train, y_train, validation_split=0.3, epochs=epochs, batch_size=bs)

# Plot train and validation loss 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show(); plt.close()

# Model evaluation
test_loss, test_acc = model.evaluate(x_test, y_test)

# Print results
print('test loss:', test_loss)
print('test accuracy:', test_acc)


Train the 2 classes model

In [0]:
# Define the optimizer and compile the model
model_oe.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model, iterating on the data in batches of 64 samples
history_oe = model_oe.fit(x_train_oe, y_train_oe, validation_split=0.3, epochs=epochs, batch_size=bs)

# Plot train and validation loss 
plt.plot(history_oe.history['loss'])
plt.plot(history_oe.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show(); plt.close()

# Model evaluation
test_loss, test_acc = model_oe.evaluate(x_test_oe, y_test_oe)

# Print results
print('test loss:', test_loss)
print('test accuracy:', test_acc)


## Predict on Test Set


In [0]:
y_pred = model.predict(x_test)
y_pred_oe = model_oe.predict(x_test_oe)

# Confusion Matrix
cm = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(np.round(y_pred),axis=1))
cm_oe = confusion_matrix(np.argmax(y_test_oe,axis=1), np.argmax(np.round(y_pred_oe),axis=1))

labels = ['class ' + str(i) for i in range(num_of_clss)] 
labels_oe = ["even", "odd"]
plot_confusion_matrix(cm,labels,title='Confusion Matrix',normalize=True)
plot_confusion_matrix(cm_oe,labels_oe,title='Confusion Matrix ODD/EVEN',normalize=True)


# Summarize

In [0]:
# Summerize the model arhiteture and parameters
model.summary()
model_oe.summary()


# Plot examples from 2 classes classification

In [0]:
# Plot example batch for odd-even classification  
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.imshow(x_train_oe[i], cmap='Greys', interpolation='none')
  plt.title("Class {}".format(y_train_oe[i]))

# Compute ROC curve for model

In [0]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
# Compute ROC curve and ROC area for each class

for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test_oe[:, i], y_pred_oe[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_oe.ravel(), y_pred_oe.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


# Plot ROC curve for model

In [0]:
plt.figure()
lw=1 # Line Width
plt.plot(fpr[i], tpr[i], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[i])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()